# How to use Amazon Forecast

Helps advanced users start with Amazon Forecast quickly. 
Prerequisites: 
[AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) . For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Table Of Contents
* [Setting up](#setup)
* [Test Setup - Running first API](#hello)
* [Forecasting Example with Amazon Forecast](#forecastingExample)

## Set up Preview SDK<a class="anchor" id="setup"></a>

In [ ]:
# Configures your AWS CLI to now understand our up and coming service Amazon Forecast
!aws configure add-model --service-model file://forecastquery-2018-06-26.normal.json --service-name forecastquery
!aws configure add-model --service-model file://forecast-2018-06-26.normal.json --service-name forecast

In [ ]:
import boto3
from time import sleep

In [ ]:
session = boto3.Session(region_name='us-west-2') #us-east-1 is also supported

forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

## Test Setup <a class="anchor" id="hello"></a>
Let's say Hi to the Amazon Forecast to interact with our Simple API ListRecipes. It displays the Global recipes that could potentially be a part of your forecasting solution. 

*If this ran successfully, kudos! If there are any errors at this point runing the following list_recipes, please send your questions to the support forum https://forums.aws.amazon.com/forum.jspa?forumID=327 


In [ ]:
forecast.list_recipes()

## Forecasting with Amazon Forecast<a class="anchor" id="forecastingExample"></a>
### Preparing your Data

In Amazon Forecast , a dataset is a file which contains data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

For this exercise, we use the individual household electric power consumption dataset. (Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.) We aggregate the usage data hourly. 

# Data Type

Amazon forecast can import data from Amazon S3. We first explore the data locally to see the fields

In [ ]:
import pandas as pd
df = pd.read_csv("data/item-demand-time.csv", dtype = object)
df.head(3)

Now upload the data to S3.

In [ ]:
s3 = session.client('s3')

In [ ]:
accountId = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
bucketName = 'amazon-forecast-%s-data'%accountId # use your own bucket

In [ ]:
s3.upload_file(s3.upload_file(Filename="./data/item-demand-time.csv", Bucket=bucketName, Key="data/item-demand-time.csv"))

In [ ]:
roleArn = 'arn:aws:iam::%s:role/amazonforecast'%accountId

### CreateDataset

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [CUSTOM](https://docs.aws.amazon.com/forecast/latest/dg/custom-domain.html) domain with 3 required attributes `timestamp`, `target_value` and `item_id`. 

In [ ]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [ ]:
project = 'workshop'
datasetName= project+'_ds'
datasetGroupName= project +'_gp'
s3DataPath = "s3://"+bucketName+"/data"

In [ ]:
datasetName

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DataFormat='CSV',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    TimeStampFormat=TIMESTAMP_FORMAT,
                    Schema = schema
                   )

In [ ]:
forecast.describe_dataset(DatasetName=datasetName)

In [ ]:
forecast.create_dataset_group(DatasetGroupName=datasetGroupName,RoleArn=roleArn,DatasetNames=[datasetName])

If you have an existing datasetgroup, you can update it

In [ ]:
forecast.describe_dataset_group(DatasetGroupName=datasetGroupName)

### Create Data Import Job
Brings the data into Amazon Forecast system ready to forecast from raw data. 

In [ ]:
ds_import_job_response=forecast.create_dataset_import_job(DatasetName=datasetName,Delimiter=',', DatasetGroupName =datasetGroupName ,S3Uri= s3DataPath)


In [ ]:
ds_versionId=ds_import_job_response['VersionId']
print(ds_versionId)

In [ ]:
forecast.update_dataset_group(DatasetGroupName=datasetGroupName, DatasetNames = [datasetName])

Check the status of dataset, when the status change from **CREATING** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**.

In [ ]:
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetName=datasetName,VersionId=ds_versionId)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'FAILED':
        sleep(30)
    else:
        break

In [ ]:
forecast.describe_dataset_import_job(DatasetName=datasetName,VersionId=ds_versionId)

### Recipe

In [ ]:
recipesResponse=forecast.list_recipes()
recipesResponse

Get details about each recipe.

In [ ]:
forecast.describe_recipe(RecipeName='forecast_MQRNN')

### Create Solution with customer forecast horizon

Forecast horizon is how long in future the forecast should be predicting. For weekly data, a value of 12 means 1 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
predictorName= project+'_mqrnn'

In [ ]:
forecastHorizon = 24

In [ ]:
createPredictorResponse=forecast.create_predictor(RecipeName='forecast_MQRNN',DatasetGroupName= datasetGroupName ,PredictorName=predictorName, 
  ForecastHorizon = forecastHorizon)

In [ ]:
predictorVerionId=createPredictorResponse['VersionId']

In [ ]:
forecast.list_predictor_versions(PredictorName=predictorName)

Check the status of solutions, when the status change from **CREATING** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorName=predictorName,VersionId=predictorVerionId)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'FAILED':
        sleep(30)
    else:
        break

### Get Error Metrics

In [ ]:
forecastquery.get_accuracy_metrics(PredictorName= predictorName)

### Deploy Predictor

In [ ]:
forecast.deploy_predictor(PredictorName=predictorName)

In [ ]:
deployedPredictorsResponse=forecast.list_deployed_predictors()

In [ ]:
while True:
    deployedPredictorStatus = forecast.describe_deployed_predictor(PredictorName=predictorName)['Status']
    if deployedPredictorStatus != 'ACTIVE' and deployedPredictorStatus != 'FAILED':
        sleep(30)
    else:
        break

### Get Forecast

When the solution is deployed and forecast results are ready, you can view them. 

In [ ]:
forecastResponse = forecastquery.get_forecast(
    PredictorName=predictorName,
    Interval="hour",
    Filters={"item_id":"client_12"}
)

# Export Forecast

You can batch export forecast to s3 bucket. To do so an role with s3 put access is needed. 

In [ ]:
forecast.create_forecast_export_job(ForecastId=forecastId, OutputPath={"S3Uri": s3DataPath,"RoleArn":roleArn})